In [ ]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import create_table_leanx as ctl


import helpers
#for Castlelight networking use case
import values_Castlelight as values
#for Sourcream use case
# import values
import master_data, text_data, sales_doc_data

In [ ]:
# master tables
for method in (
     master_data.users,
     master_data.customers_and_vendors, 
     master_data.plants, 
     master_data.materials, 
     master_data.material_support,
     master_data.routes,
     master_data.company_codes
 ):
     table_dict = method()    
     for k, v in table_dict.items():
         table_name = k.split('_')[0]
         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
         all_cols_req = ctl.clean_columns(all_cols,table_name)
         df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
         df.to_csv(f'data/OCPM/master/{table_name}.csv', index=False)

In [ ]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.doc_types, 
    text_data.organization,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        all_cols_req = ctl.clean_columns(all_cols,table_name)
        df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/text/{table_name}.csv', index=False)

In [ ]:
MARC = pd.read_csv('data/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/OCPM/master/MAKT.csv')
KNB1 = pd.read_csv('data/OCPM/master/KNB1.csv')
KNA1 = pd.read_csv('data/OCPM/master/KNA1.csv')

all_prices = {}
all_availabilities = {}
all_material_groups = {}

# get price by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_availabilities[nr] = attr['availability']

def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

def get_params():
    #company_code = random.choice(list(values.om_company_codes.keys()))
    #emphasis on EMEA company codes / plants
    company_code = random.choices(list(values.om_company_codes.keys()),[10, 60, 65, 20, 75, 15, 55, 30, 35, 60, 15, 60, 13, 20, 15, 10, 35, 65],k=1)[0]
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    # all_matnrs = MARC[MARC['WERKS'] == 'PL01']['MATNR'].unique()
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))
    # quantities = [random.randint(12, 60)*12 for _ in range(len(matnrs))] # x dozens of everything
    quantity_factor = random.randint(5, 7) if values.om_plants[plant]['high_value'] else random.randint(2, 4)
    quantities = [random.randint(3, 6)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    prices = []
    availabilities = []
    delivery_status_boundaries = []
    
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]])
        availabilities.append(all_availabilities[matnrs[i]])
        a = all_availabilities[matnrs[i]]
        #[Probability of being late, Prabability of being late + Prob of being on Time]
        # Prob of late = 0.7-0.65*a, Prob of On-Time = 0.25+0.55*a, Prob of Early = 0.05 +0.1*a
        # -> Boundaries CASE WHEN random number < Prob of late Then LATE, CASE WHEN random number > Prob of Late AND <Prob of Late + Prob of ON-Time THEN ON-Time ELSE Early END
        delivery_status_boundaries.append([0.9-0.7*a,0.95-0.1*a])
    
    #sales_org=random.choice(list(values.om_sales_orgs.keys()))
    sales_org=random.choice(values.om_plants[plant]['sales_orgs'])
    sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    doc_type_number = random.random()
    # sales_doc_type= 'ZDIR' if random.random() < 0.7 else random.choice(list(values.om_sales_doc_types.keys()))
    sales_doc_type = ('ZDIR' if (doc_type_number < 0.4) else
                      'ZOR' if (doc_type_number < 0.8) else 
                      random.choice(list(values.om_sales_doc_types.keys()))
                      ) 
    mapping_salesdoc = {'ZOR': '40', 'ZDLR': '20', 'ZDIR': '10', 'ZDIS': '30','ZEXP': '50'}
    distribution_channel=mapping_salesdoc[sales_doc_type]
    kunnr = random.choice(list(KNB1[KNB1['BUKRS'] == company_code]['KUNNR']))
    customer_name = KNA1[KNA1['KUNNR'] == kunnr]['NAME1'].values[0]
    payment_term = random.choice(list(KNB1[KNB1['KUNNR'] == kunnr]['ZTERM']))
    credit_risk = values.om_customers[customer_name]['credit_risk']
    
    automation_rate = values.om_sales_orgs[sales_org]['Automation_rate'][distribution_channel]
    #sales_doc_type=random.choice(list(values.om_sales_doc_types.keys()))
    requested_dev_date = random.randint(7, 17)
    total = np.sum([prices[i]*quantities[i] for i in range(len(prices))])
# --------------------------------------------------------------------------

    item_delivery_status = []
    for i in range(len(delivery_status_boundaries)):
        late_bound, ot_bound = delivery_status_boundaries[i][0], delivery_status_boundaries[i][1]
        r = random.random()
        # if r < late_bound or (r < 0.4 and requested_dev_date <= 11):
        if (r < 0.7 and requested_dev_date <= 11):
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif (r < 0.4 and requested_dev_date >= 11):
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif r < late_bound:
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif r >= late_bound and r<ot_bound:
            item_delivery_status.append({'status': 'ot', 'prob': round(r-late_bound,2)})
        else:
            item_delivery_status.append({'status': 'early', 'prob': round(r-ot_bound,2)})
# --------------------------------------------------------------------------
        # time distribution
    pr_req_years = [2023, 2024]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]
    if pr_req_year == 2023:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 5, 6, 7, 10])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    else:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 8])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(9, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    
    params = {
        'kunnr': kunnr,
        'credit_risk': credit_risk,
        'payment_term': payment_term,
        'company_code': company_code,
        'plant': plant,
        'matnrs': matnrs,
        'quantities': quantities,
        'prices': prices,
        'sales_org': sales_org,
        'sales_office': sales_office,
        'distribution_channel': distribution_channel,
        'availabilities': availabilities,
        'sales_doc_type': sales_doc_type,
        'total': total,
        'automation_rate': automation_rate,
        'item_delivery_status': item_delivery_status,
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
        'requested_dev_date' : requested_dev_date
    }

    return params

In [ ]:
# sales order tables
sales_doc_tables = {
    'VBAK_json': {},
    'VBKD_json': {},
    'VBUK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'EKBE_json': {}, 
    'MSEG_json': {},
    'VBRK_json': {},
    'VBRP_json': {},
    'BKPF_json': {},
    'BSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'JCDS_json': {},
    'VBFA_json': {},
}


    # transition_prob = np.array([
    #     [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Approve Sales Order
    #     [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Generate Delivery Document
    #     [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Pick Items
    #     [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Post Goods Issue
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Send Invoice
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Recieve Delivery Confirmation
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Clear Invoice
        
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Reject Sales Order
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.25], # Set Credit Block
    #     [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Credit Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00,0.00, 0.35], # Set Delivery Block
    #     [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Delivery Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85, 0.00, 0.00,0.00, 0.15], # Set Sales Order Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Release Sales Order Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85,0.00, 0.15], # Set Customer Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Customer Billing Block
    #     [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Change Payment Term
        
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00] # Terminate
item_status_early = {"Early":0, "Early works": 0}
item_status_late = {"Late":0, "Late works": 0}
item_status_ot = {"ot":0, "ot works": 0}

for i in range(27292):
    transition_prob = np.array([
      # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ] 
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 0 Approve Sales Order
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 1 Generate Delivery Document
        [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 2 Pick Items
        [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 3 Post Goods Issue
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 4 Send Invoice
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 5 Recieve Delivery Confirmation
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 6 Clear Invoice
         
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 7 Reject Sales Order
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.25], # 8 Set Credit Block
        [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 9 Release Credit Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35], # 10 Set Sales Order Delivery Block
        [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 11 Release Sales Order Delivery Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 12 Set Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 13 Release Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.61, 0.00, 0.00, 0.05], # 14 Set Customer Billing / Delivery Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 15 Release Customer Billing / Delivery Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 16 Change Payment Term
        [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 17 Change Quantity / Price

        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 18 Terminate

    ])

    """
    Steps map
    Starts at:  Create Sales Order
    0: Approve Sales Order
    1: Generate Delivery Document
    2: Pick Items
    3: Post Goods Issue
    4: Send Invoice
    5: Recieve Delivery Confirmation
    6: Clear Invoice

    7: Reject Sales Order
    8: Set Credit Block
    9: Release Credit Block
    10: Set Delivery Block
    11: Release Delivery Block
    12: Set Sales Order Billing Block
    13: Release Sales Order Billing Block
    14: Set Customer Billing Block
    15: Release Customer Billing Block
    16: Change Payment Term
    17: Change Quantity

    18: Terminate Process
    """

    params=get_params()
    #latest_date = helpers.generate_random_date(start_date=datetime(2022, 1, 1), end_date=datetime(2023, 12, 31))
    latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
    so_created_date = latest_date
    latest_time = helpers.generate_random_time()
    so_created_by = get_user_name(0.9*params['automation_rate'])
    reqested_delivery_date= latest_date + timedelta (days = params['requested_dev_date'])
    # print(params['total'])
    # reqested_delivery_date=helpers.add_random_days(min_days=7, max_days=17, current_date=latest_date)

    sd = sales_doc_data.SalesAndDistribution(
        vbeln=f'{str(uuid.uuid4())[-11:]}', #WARNING CHANGED FROM 15 TO 11 FOR PAYMENT TERMS CHANGE 
        params=params,
        start_date=latest_date
    )

    sd.create_sales_order(
        reqested_delivery_date=reqested_delivery_date,
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        erdat=latest_date,
        ernam=so_created_by,
        atime=latest_time
    )
    # latest_date += helpers.UPTO_WEEK()
    latest_time = helpers.add_random_hours(1, latest_time)
    

    # Define the deviation of days between scheduled issue date VBEP.EDATU and actual issue date MSEG.CPUDT_MKPF
    delivery_date_deviation = []
    for i in range(len(params['matnrs'])):
        days_deviation = helpers.UPTO_MONTH()*params['item_delivery_status'][i]['prob']
        days_temp = round(days_deviation.total_seconds()/(24*3600))
        days_deviation = days_temp #timedelta(days=days_temp)
        if params['item_delivery_status'][i]['status'] == 'late':
            scheduled_date = min(-1, -days_deviation)
        elif params['item_delivery_status'][i]['status'] == 'early':
            scheduled_date  = max(1, days_deviation)
        else:
            scheduled_date = 0
        delivery_date_deviation.append(scheduled_date)
        


    # decide on apprve or reject sales order
    def nex_step():
        stepy = 0
        if so_created_by != 'BATCH_JOB':
            if random.random() < 0.4:
                stepy = 7 # Reject Sales Oreder for inaccurate information
        # Quantity Change
        elif np.average(params['availabilities']) < 0.6:
            if random.random() < 0.1:
                stepy = 17

        return stepy # Approve Reject or Change
    
    step = nex_step()
    while step != 18: # until termination
        if step == 0:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.approve_sales_order(
                usnam=get_user_name(0.9*params['automation_rate']),
                udate=latest_date,
                atime=latest_time
            )
            
            # Set Credit Block
            if params['credit_risk'] > 0.75:
                transition_prob[step][8] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Customer Billing / Delivery Block
            elif params['credit_risk'] > 0.6:
                transition_prob[step][14] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Change Payment Term
            elif random.random() > 0.95:
                transition_prob[step][16] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)
        elif step == 1:
            sd.generate_delivery_document(
                ernam=get_user_name(0.9*params['automation_rate']), 
                erdat=latest_date,
                planned_delivery_date=latest_date + helpers.UPTO_WEEK(),
                picking_date=None,
                delivery_date=None,
                confirmation_date=None,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(3, latest_time)

            # Delivery blocks
            if np.average(params['availabilities']) < 0.6:
                latest_time = helpers.add_random_hours(1, latest_time)
                transition_prob[step][10] = 0.5
            else:
                latest_time = helpers.add_random_hours(1, latest_time)
        elif step == 2:
            sd.pick_items(
                usnam=get_user_name(0.4*params['automation_rate']), 
                udate=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 3:
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(3, 6, latest_date)
            sd.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.6*params['automation_rate']),
                delivery_date_deviation = delivery_date_deviation,
                atime=helpers.generate_random_time()         
            )
# ------ Uncomment snippet to understand early and late deliveries
            # for i, d in enumerate(delivery_date_deviation):
            #     if d>0:
            #         item_status_early["Early"]+=1
            #         if latest_date+timedelta(days=delivery_date_deviation[i]) > (so_created_date + timedelta(days=1)):
            #             item_status_early["Early works"]+=1
            #         else:
            #             print(f'Deviation from delivery date is {d} days. Min Date is {so_created_date + timedelta(days=1)} new date should be {latest_date+timedelta(days=d)}, send goods date {latest_date}')
            #     elif d<0:
            #         item_status_late["Late"]+=1
            #         if latest_date+timedelta(days=delivery_date_deviation[i]) > (so_created_date + timedelta(days=1)):
            #             item_status_late["Late works"]+=1
            #         else:
            #             print(f'Deviation from delivery date is {d} days. Min Date is {so_created_date + timedelta(days=1)} new date should be {latest_date+timedelta(days=d)}, send goods date {latest_date}')
            #     else:
            #         item_status_ot["ot"]+=1
# ----------------------------------------------------------------------------------
        # Sales Order Billing Block
            if params['total'] > 450000:
                transition_prob[step][12] = 0.3
                latest_time = helpers.add_random_hours(12, latest_time)
                # latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
                latest_date = helpers.add_random_days(0, 7, latest_date)   
            else:
                latest_time = helpers.add_random_hours(2, latest_time)
                # latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 4:
            sd.create_invoice(
                ernam=get_user_name(0.8*params['automation_rate']),
                erdat=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 5:
            sd.delivery_confirmation(
                usnam=get_user_name(0.8*params['automation_rate']), 
                udate=latest_date
            )
            latest_time = helpers.add_random_hours(1, latest_time)
        
            if get_time_consumption(start_date=so_created_date, planned_target_date=reqested_delivery_date, latest_date=latest_date) > 1.25:
                latest_date = helpers.add_random_days(0, 14, latest_date)
            else:
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 6:
            sd.clear_debit_invoice(
                cpudt=latest_date,
                usnam=get_user_name(0.4*params['automation_rate']),
                cleared_date=latest_date + helpers.UPTO_WEEK(),
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        
        elif step == 7:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.reject_sales_order(
                udate=latest_date,
                usnam=get_user_name(0.9*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 8:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.set_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.6*params['automation_rate'])
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(1, 15, latest_date)
        elif step == 9:
            sd.release_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 10:
            sd.set_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.7*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(1, 30, latest_date)
        elif step == 11:
            sd.release_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 12:
            sd.set_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.9*params['automation_rate']), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(1, 7, latest_date)
        elif step == 13:
            sd.release_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(4, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 14:
            if params['credit_risk'] > 0.67:
                sd.set_customer_billing_block(
                    udate=latest_date,
                    usnam=get_user_name(0.7*params['automation_rate']), 
                )
            else:
                sd.set_customer_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.7*params['automation_rate']), 
                )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 14, latest_date)
        elif step == 15:
            if params['credit_risk'] > 0.67:
                sd.release_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.3*params['automation_rate']), 
                )
            else:
                sd.release_customer_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.3*params['automation_rate']),
                )
            latest_time = helpers.add_random_hours(5, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 16:
            sd.change_payment_term(
                udate=latest_date,
                usnam=get_user_name(0.3*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)

        elif step == 17:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            if random.random() < 0.5:
                old_quantities = params['quantities']
                new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))
                new_quanity_quantities = [round(old_quantities[j]*(1+random.random())) for j in new_quantity_lines]
                sd.change_quantity( # likely goes to 'Send PO'
                    udate=latest_date,
                    usnam=get_user_name(0.1*params['automation_rate']),
                    line_numbers=new_quantity_lines,
                    line_quantities=new_quanity_quantities
                )
            else:
                old_prices = params['prices']
                new_price_lines = random.sample(range(len(old_prices)), random.randint(0, len(old_prices)))
                new_prices = [round(old_prices[j]*(random.uniform(0.5, 1.5)), 2) for j in new_price_lines]
                sd.change_price( # likely goes to 'Send PO'
                    udate=latest_date,
                    usnam=get_user_name(0.1*params['automation_rate']),
                    line_numbers=new_price_lines,
                    line_prices=new_prices
                ) 
            latest_date += helpers.UPTO_WEEK()
        # update transition matrix to sum to 1
        transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
        step = np.random.choice(19, p=transition_prob[step])

    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

In [ ]:
# item_status_early
# item_status_late
# BILLING BLOCK ROOT CAUSE ZERO PRICE SETTING TO ADJUST %

for k, v in sales_doc_tables['VBAP_json'].items():
    if v['FAKSP'] == '01':
        v['NETWR'] = 0.00
release_list = []
for k, v in sales_doc_tables['CDPOS_json'].items():
    if v['FNAME'] == 'FAKSP' and v['VALUE_NEW'] == None:
        release_list.append(v['TABKEY'][2:13])

for k, v in sales_doc_tables['VBAP_json'].items():
    if v['VBELN'] in release_list:
        v['NETWR'] = 0.00
        # print(v['NETWR'])

# DELIVERY BLOCK ROOT CAUSE INCOMPLETE ORDER ADJUST SHIPPING TERMS. TO ADJUST % 
for k, v in sales_doc_tables['VBAK_json'].items():
    if v['LIFSK'] == '01':
        v['VSBED'] = None

release_list2 = []
for k, v in sales_doc_tables['CDPOS_json'].items():
    if v['FNAME'] == 'LIFSK' and v['VALUE_NEW'] == None:
        release_list2.append(v['TABKEY'][2:13])

for k, v in sales_doc_tables['VBAK_json'].items():
    if v['VBELN'] in release_list2:
        v['VSBED'] = None




In [ ]:
# insufficient material root cause - add schedule lines in VBEP

lowavailmats = ['CLMAT001', 'CLMAT002', 'CLMAT003', 'CLMAT004', 'CLMAT120', 'CLMAT121', 'CLMAT122', 'CLMAT123', 'CLMAT124', 'CLMAT130', 'CLMAT131', 'CLMAT132', 'CLMAT133', 'CLMAT134']
orderitemslist = []
for k, v in sales_doc_tables['VBAP_json'].items():
    if v['MATNR'] in lowavailmats:
        orderitemslist.append(f'{v['VBELN']}{v['POSNR']}')

# for k, v in sales_doc_tables['VBEP_json'].items(): 
#   print(v)
extrarows = {}

for k, v in sales_doc_tables['VBEP_json'].items(): 
    if f'{v['VBELN']}{v['POSNR']}' in orderitemslist:
        new_quant = v['BMENG'] - 5
        v['BMENG'] = new_quant
        temp_vbeln = f'{str(uuid.uuid4())[-11:]}'
        extrarows[temp_vbeln] = {
            "BMENG": 5,
            "EDATU": v['EDATU'],
            "ETENR": v['ETENR']+1,
            "MANDT": v['MANDT'],
            "MBDAT": None, 
            "MEINS": v['MEINS'],
            "POSNR": v['POSNR'],
            "VBELN": v['VBELN'],
            "WADAT": None
            }
        
sales_doc_tables['VBEP_json'].update(extrarows)

In [ ]:
# # DELIVERY BLOCK CUSTOMER LEVEL 

# KNA1 = pd.read_csv('data/OCPM/master/KNA1.csv')
# # print(KNA1['LIFSD'])
# def setcustblock(x):
#     return '02'

# cust_block = KNA1['LIFSD']
# new_cust_block = cust_block.apply(setcustblock)
# KNA1['LIFSD'] = new_cust_block

# # KNA1.loc[KNA1, 'LIFSD'] = '01'

# KNA1.to_csv(f'data/OCPM/master/KNA1.csv', index=False)
# params=get_params()
# # params['prices']
# old_prices = params['prices']
# print(old_prices)
# random.sample(range(len(old_prices)), random.randint(0, len(old_prices)))

            # new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))

In [ ]:
# adding customer block to KNA1 after the fact where block was not removed. 

blocklist = []

for k, v in sales_doc_tables['CDPOS_json'].items():
    if v['FNAME'] == 'FAKSD' and v['TABNAME'] == 'KNA1' and v['VALUE_NEW'] == '02':
        blocklist.append(v['TABKEY'])


for i in blocklist:
    for k, v in sales_doc_tables['CDPOS_json'].items():
        if i == v['TABKEY'] and v['VALUE_OLD'] == 'Block Removed':
             blocklist.remove(i)


KNA1 = pd.read_csv('data/OCPM/master/KNA1.csv')
  
for index, row in KNA1.iterrows():
    if f'CL{row['KUNNR']}' in blocklist:
        # print('true')
        # print(row['KUNNR'])
        KNA1.loc[index,'FAKSD'] = '02'

KNA1.to_csv(f'data/OCPM/master/KNA1.csv', index=False)



In [ ]:
for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/OCPM/sales-document/{table_name}.csv', index=False)